### download the model checkpoint and configure it so that you can fine-tune it later on

In [ ]:
!pip install -q transformers flash_attn timm einops peft
!pip install -q roboflow git+https://github.com/roboflow/supervision.git

### Imports

In [ ]:
# @title Imports
import cv2
import io
import os
import re
import json
import torch
import html
import base64
import itertools

import numpy as np
import supervision as sv

from google.colab import userdata
from IPython.core.display import display, HTML
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AdamW,
    AutoModelForCausalLM,
    AutoProcessor,
    get_scheduler
)
from tqdm import tqdm
from typing import List, Dict, Any, Tuple, Generator
from peft import LoraConfig, get_peft_model
from PIL import Image
from roboflow import Roboflow

In [ ]:
CHECKPOINT = "microsoft/Florence-2-base-ft"
REVISION = 'refs/pr/6'
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = AutoModelForCausalLM.from_pretrained(CHECKPOINT, trust_remote_code=True, revision=REVISION).to(DEVICE)
processor = AutoProcessor.from_pretrained(CHECKPOINT, trust_remote_code=True, revision=REVISION)

In [ ]:
task = "<DETAILED_CAPTION>"
text = "<DETAILED_CAPTION>"


In [ ]:
cap = cv2.VideoCapture('/content/test_video.mp4')

frame_number = 1
frame_and_caption = {}  # Initialize dictionary to store frame numbers and captions
frame_count = 0

while True:
    ret, frame = cap.read()

    if not ret:
        break  # Exit the loop if no more frames are available

    
    frame_count += 1
    if frame_count % 10 == 0:
     
        inputs = processor(text=text, images=frame, return_tensors="pt").to(DEVICE)
        generated_ids = model.generate(
              input_ids=inputs["input_ids"],
              pixel_values=inputs["pixel_values"],
              max_new_tokens=1024,
              num_beams=3
          )
        
        generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
          
          # Assuming 'task' and 'image_size' are defined earlier
        response = processor.post_process_generation(generated_text, task=task, image_size=(frame.shape[1], frame.shape[0]))
          
          # Store the generated caption for the current frame
        if '<DETAILED_CAPTION>' in response:
              frame_and_caption[frame_number] = response['<DETAILED_CAPTION>']

        frame_number += 1
        
    
    # Break the loop on pressing 'Esc'
    key = cv2.waitKey(1)
    if key == 27:
      break

cap.release()
cv2.destroyAllWindows()



In [ ]:
# Print the stored frame and caption data
print(frame_and_caption)